############################################################################################################################################################

<strong>Team 2 Group Project<strong>

<strong>Data Optimization & Python Group Assignment<strong>

Submission by: Alexander Beveridge, Belén Flecha, Jinina Rei Garcia, Luis Mendez, Darshil Panchal.

    
<strong>STEP 1 - CREATING DATAFRAME<strong>
    
In this Data Frame, we included key information on the problem we were trying to solve. Firstly, we included the amount of production runs that it would take to achieve at least 90% demand in each region for the total amount of product. This was calculated by getting the maximum amount of units per run for each product (90 for original and 70 for flavor blast) and kept running the product until we reached above 90% satisfaction for demand. For all regions this was 100% demand met, however for South Central, this was 98% and adjust the revenue due to this factor. 
    
Secondly, we included the revenue that was made after demand is met. As explained, this was all maximum revenue apart from South Central whereby only 98% of demand was met to minimize the amount of production runs. This was in USD as per the case. 

Lastly, we included the associated cost for the entry port regions. This cost is a reference to the cost to set up the production factory in Thailand. Note that there are two regions that are not points of entry and therefore have no associated costs with setting up the production factory - the cost for these will be included with the entry port it is connected to (South Central is connected to North Central and would therefore assume it's associated costs as a point of entry). 
    
<strong>STEP 2 - CREATING LP MODEL<strong>
    
The second step was to create an LP Model with the Data Frame we had created; We used PuLP to create the model. First, we stated our decision variables. These were the regions in which we wanted to enter the market. These were defined as abbreviations (e.g. NE = North East), and we created the decision variables in binary format, just like our excel model, in order for our model to recognize our constraints. Secondly, we multiplied the production runs in the Data Frame by the binary output of our decision variables ("solver"). If the binary value equaled 1, then the model would switch on the production run within the confine of the maximum 5 constraint. We used the Data Frame locater to input the production runs.
    
Further to this, we inputted our objective function, which was to maximize revenue based on the regions and unit revenue that we had within the Data Frame. We used the locater once again but this time for revenue and multiplied it by the decision variables as decision variables are essential within an objective function.
    
We then listed our constraints. These were comprised of two parts. First, the constraint that says that the sum of the production run * decision variable (solver) could not exceed 5 runs. The second part was to list routes that were not possible. For example, we could not have NE - SC - WC as a route, and therefore limited its binary output to 2 or under (if the solver tried to add up all three as '1' values, this would go against the constraint). 
    
Finally, we printed the LP model and used the solver to give us our desired output. 
    
<strong>CONCLUSIONS<strong>    
    
1. Our first conclusion is that the optimal route for the production of both types of popcorn to satisfy demand would be <strong>North Central - South Central<strong>.
    
    Our output from the LP Model identified the 4th column and 5th column as '1.0' and the rest as '0.0'. This binary allowed us to identify the regions we would select for the route and which ones not to consider. As North Central was the only point of entry, the route would start there and end in South Central.
    
    
2. Revenue from the route is calculated as the Optimum/Objective Value. This was USD3,637,708. Therefore, the optimal revenue we can get from satisfying popcorn demand in China, taking into account the constraints that were identified, was this value of <strong>USD3,637,708<strong>.
    
    
3. We can conclude that our cost would come from building a factory in Chiang Rai. We took on the worst possible outcome for this cost, presuming that weather conditions are bad and that the factory is delayed due to failed inspection. This was actually still more profitable than the 2nd best route (YRD - NE) and in addition, the 2nd best route would conflict with the governmental rules of heavy industry within that region. So, even with a higher cost through the identified problems, the production region of Chiang Mai, alongside the selected route of North Central and South Central, was still the most profitable. 
    
    
<strong>Updated and Submitted: 2022-02-18<strong>

############################################################################################################################################################


In [4]:
## ## ## STEP 1 - CREATING DATAFRAME FROM EXCEL ## ## ##

# Importing Pandas as pd to create DataFrame
import pandas as pd

# Creating DataFrame
df = pd.DataFrame([[1, 525000, 11252161], [4, 3090000, 24062500], [3, 1865000], [2, 1650000, 12248001], [3, 1987708], [1, 1125000, 10303601]], \
     index = ['North East', 'YRD', 'South East', 'North Central', 'South Central', 'Western China'], \
     columns = ['Production Runs', 'Revenue', 'Cost Association'])

# Fill NA's in DataFrame with zero - these are not entry regions so cost is not associated with the two NA regions
LP_df = df.fillna(0)

# Print DataFrame
print(LP_df)

## ## ## STEP 2 - CREATING LP MODEL USING DATAFRAME ## ## ##

# Importing pulp as p to create LP Model
import pulp as p
  
# Creating an LP Maximization problem
Lp_prob = p.LpProblem('Problem', p.LpMaximize) 
  
# Creating Decision Variables 
NE = p.LpVariable("NE", lowBound = 0, upBound = 1, cat = 'Binary')   # Create a variable NE >= 0 and binary
YRD = p.LpVariable("YRD", lowBound = 0, upBound = 1, cat = 'Binary')   # Create a variable YRD >= 0 and binary
SE = p.LpVariable("SE", lowBound = 0, upBound = 1, cat = 'Binary')   # Create a variable SE >= 0 and binary
NC = p.LpVariable("NC", lowBound = 0, upBound = 1, cat = 'Binary')   # Create a variable NC >= 0 and binary
SC = p.LpVariable("SC", lowBound = 0, upBound = 1, cat = 'Binary')   # Create a variable SC >= 0 and binary
WC = p.LpVariable("WC", lowBound = 0, upBound = 1, cat = 'Binary')   # Create a variable WC >= 0 and binary

# Runs * Solver
NE_RUN = LP_df.loc['North East', 'Production Runs'] * NE
YRD_RUN = LP_df.loc['YRD', 'Production Runs'] * YRD
SE_RUN = LP_df.loc['South East', 'Production Runs'] * SE
NC_RUN = LP_df.loc['North Central', 'Production Runs'] * NC
SC_RUN = LP_df.loc['South East', 'Production Runs'] * SC
WC_RUN = LP_df.loc['Western China', 'Production Runs'] * WC

# Objective Function (MAX Profit)
Lp_prob += LP_df.loc['North East', 'Revenue'] * NE + LP_df.loc['YRD', 'Revenue'] * YRD \
           + LP_df.loc['South East', 'Revenue'] * SE + LP_df.loc['North Central', 'Revenue'] * NC \
           + LP_df.loc['South Central', 'Revenue'] * SC + LP_df.loc['Western China', 'Revenue'] * WC 
  
# Constraints:
Lp_prob += (NE_RUN + YRD_RUN + SE_RUN + NC_RUN + SC_RUN + WC_RUN) <= 5 # Production runs must be maximum 5
Lp_prob += NE + SC + WC <= 2 # Region route not possible
Lp_prob += YRD + SC + WC <= 2 # Region route not possible
Lp_prob += NE + SE + WC <= 2 # Region route not possible
Lp_prob += NE + WC <= 1 # Region route not possible
Lp_prob += YRD + WC <= 1 # Region route not possible
Lp_prob += SE + WC <= 1 # Region route not possible
Lp_prob += NE + SC <= 1 # Region route not possible
Lp_prob += YRD + SC <= 1 # Region route not possible

  
# Printing the problem
print(Lp_prob)
  
status = Lp_prob.solve() # Solver
print(p.LpStatus[status]) # Solution Status
  
# Printing final solution
print(p.value(NE), p.value(YRD), p.value(SE), p.value(NC), p.value(SC), p.value(WC), p.value(Lp_prob.objective))

# Optimal Value = $3,637,708.00
# Binary Route = 0.0, 0.0, 0.0, 1.0, 1.0, 0.0
# Route = North Central, South Central

               Production Runs  Revenue  Cost Association
North East                   1   525000        11252161.0
YRD                          4  3090000        24062500.0
South East                   3  1865000               0.0
North Central                2  1650000        12248001.0
South Central                3  1987708               0.0
Western China                1  1125000        10303601.0
Problem:
MAXIMIZE
1650000*NC + 525000*NE + 1987708*SC + 1865000*SE + 1125000*WC + 3090000*YRD + 0
SUBJECT TO
_C1: 2 NC + NE + 3 SC + 3 SE + WC + 4 YRD <= 5

_C2: NE + SC + WC <= 2

_C3: SC + WC + YRD <= 2

_C4: NE + SE + WC <= 2

_C5: NE + WC <= 1

_C6: WC + YRD <= 1

_C7: SE + WC <= 1

_C8: NE + SC <= 1

_C9: SC + YRD <= 1

VARIABLES
0 <= NC <= 1 Integer
0 <= NE <= 1 Integer
0 <= SC <= 1 Integer
0 <= SE <= 1 Integer
0 <= WC <= 1 Integer
0 <= YRD <= 1 Integer

Optimal
0.0 0.0 0.0 1.0 1.0 0.0 3637708.0
